In [ ]:
!pip install -q opencv-python-headless

In [ ]:
from google.colab import files
uploaded = files.upload()  # select your .h5
!mv *.h5 /content/mask_detector_resnet50_tpu.h5


Saving mask_detector_resnet50_tpu.h5 to mask_detector_resnet50_tpu.h5
mv: 'mask_detector_resnet50_tpu.h5' and '/content/mask_detector_resnet50_tpu.h5' are the same file


In [ ]:
import os
print("Model found:", os.path.exists("/content/mask_model.h5"))


Model found: True


In [ ]:
import shutil
for f in uploaded:
    if f.endswith(".h5"):
        shutil.move(f, "/content/mask_model.h5")


In [ ]:
#-------------------------------
#----------------------------
#------------------------------

# ============================================
# Webcam Face Mask Detector for Colab (with upload_images_and_detect)
# ============================================

# Uncomment these if you haven’t already installed OpenCV in Colab:
# !pip install -q opencv-python-headless

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io
import matplotlib.pyplot as plt
import os
from google.colab import files


# ============================================
# 1) ColabMaskDetector Class (unchanged except for minor formatting)
# ============================================
class ColabMaskDetector:
    def __init__(self, model_path="/content/mask_head.h5"):
        # Workaround for mixed‐precision “Cast” layer if present:
        class DummyLayer(tf.keras.layers.Layer):
            def call(self, inputs):
                return inputs

        # Load the model, replacing “Cast” with DummyLayer if necessary
        self.model = load_model(
            model_path,
            custom_objects={"Cast": DummyLayer},
            compile=False
        )

        # Determine input size from the model itself
        self.img_size = self.model.input_shape[1]
        self.labels = ['Without Mask', 'With Mask']
        self.colors = {
            'Without Mask': (255, 0, 0),
            'With Mask': (0, 255, 0)
        }
        # Load Haar cascade for face detection
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        print(f"✅ Model loaded: {model_path}, input size: {self.img_size}x{self.img_size}")

    def preprocess_face(self, face):
        # Resize, convert to RGB, normalize, and add batch dimension
        face = cv2.resize(face, (self.img_size, self.img_size))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=0)
        return face

    def annotate_faces(self, frame, faces):
        # For each detected face, predict mask/no‐mask and draw a box + label
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            input_face = self.preprocess_face(face_img)
            pred = self.model.predict(input_face, verbose=0)[0]
            idx = np.argmax(pred)
            label = self.labels[idx]
            confidence = pred[idx] * 100
            color = self.colors[label]
            # Draw rectangle and put text
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(
                frame,
                f"{label}: {confidence:.1f}%",
                (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2
            )
        return frame

    def upload_images_and_detect():
        print("📁 Please select one or more image files to upload (e.g. .jpg, .png):")
        uploaded_dict = files.upload()
        if not uploaded_dict:
            print("❌ No files uploaded.")
            return

        for fname, file_bytes in uploaded_dict.items():
            try:
                # Load the uploaded image into a PIL Image
                img = Image.open(io.BytesIO(file_bytes)).convert("RGB")
                img_array = np.array(img)

                # Run face+mask detection
                annotated_img, results = detector.detect_faces_and_masks(img_array)

                # Display original and annotated side by side
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
                ax1.imshow(img_array)
                ax1.set_title(f"Original: {fname}", fontsize=14)
                ax1.axis("off")

                ax2.imshow(annotated_img)
                ax2.set_title(f"Mask Detection: {fname}", fontsize=14)
                ax2.axis("off")

                plt.tight_layout()
                plt.show()

                # Print a simple summary
                if results:
                    print(f"✅ {len(results)} face(s) detected in '{fname}':")
                    for i, r in enumerate(results, start=1):
                        label, conf = r["label"], r["confidence"]
                        status = "✓ SAFE" if label == "With Mask" else "⚠ NO MASK"
                        bbox = r["bbox"]
                        print(f"   • Face {i}: {label} ({conf*100:.1f}%) → {status} at {bbox}")
                else:
                    print(f"❌ No faces detected in '{fname}'.")

                print("\n" + "="*60 + "\n")

            except Exception as e:
                print(f"❌ Unable to process '{fname}': {e}")
                continue

    def capture_and_detect(self):
        # Javascript snippet to grab one webcam frame when “Capture” is clicked
        js = Javascript('''
        async function captureWebcam() {
          const div = document.createElement('div');
          const video = document.createElement('video');
          const btn = document.createElement('button');
          btn.textContent = 'Capture';
          div.appendChild(video);
          div.appendChild(btn);
          document.body.appendChild(div);

          const stream = await navigator.mediaDevices.getUserMedia({ video: true });
          video.srcObject = stream;
          await video.play();

          const canvas = document.createElement('canvas');
          const context = canvas.getContext('2d');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;

          const result = await new Promise((resolve) => {
            btn.onclick = () => {
              context.drawImage(video, 0, 0);
              stream.getTracks().forEach(track => track.stop());
              resolve(canvas.toDataURL('image/jpeg'));
            };
          });

          div.remove();
          return result;
        }
        captureWebcam();
        ''')

        print("📸 Waiting for webcam... Click 'Capture' when ready.")
        display(js)
        data = eval_js("captureWebcam()")
        binary = b64decode(data.split(',')[1])
        img = Image.open(io.BytesIO(binary))
        frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

        # Detect faces in the captured frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5)

        # Annotate and display
        annotated = self.annotate_faces(frame.copy(), faces)
        rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(8, 6))
        plt.imshow(rgb)
        plt.axis('off')
        plt.title("Mask Detection Result")
        plt.show()

# ============================================
# 2) Initialize detector (run just once)
# ============================================
model_path = "/content/mask_model.h5"

if not os.path.exists(model_path):
    print("❌ Model file not found. Please upload it to /content/ first.")
else:
    detector = ColabMaskDetector(model_path)
    # (Optional) Immediately run one webcam capture if desired:
    # detector.capture_and_detect()

# ============================================
# 3) New Function: upload_images_and_detect()
# ============================================
def upload_images_and_detect():
    """
    Upload one or more images from your local system, then run face+mask detection
    on each and display original vs annotated side by side.
    """
    print("📁 Please select one or more image files to upload (e.g. .jpg, .png):")
    uploaded_dict = files.upload()
    if not uploaded_dict:
        print("❌ No files uploaded.")
        return

    for fname, file_bytes in uploaded_dict.items():
        try:
            # Load uploaded bytes into a PIL Image, convert to RGB
            img = Image.open(io.BytesIO(file_bytes)).convert("RGB")
            img_array = np.array(img)  # HxWx3 BGR? Actually this is RGB

            # Convert to BGR before feeding into detector
            bgr_frame = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            gray = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2GRAY)
            faces = detector.face_cascade.detectMultiScale(gray, 1.1, 5)

            # Annotate
            annotated_bgr = detector.annotate_faces(bgr_frame.copy(), faces)
            annotated_rgb = cv2.cvtColor(annotated_bgr, cv2.COLOR_BGR2RGB)

            # Display side by side
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
            ax1.imshow(img_array)
            ax1.set_title(f"Original: {fname}", fontsize=14)
            ax1.axis("off")

            ax2.imshow(annotated_rgb)
            ax2.set_title(f"Mask Detection: {fname}", fontsize=14)
            ax2.axis("off")

            plt.tight_layout()
            plt.show()

            # Print summary text
            if len(faces) > 0:
                print(f"✅ {len(faces)} face(s) detected in '{fname}':")
                for i, (x, y, w, h) in enumerate(faces, start=1):
                    face_img = bgr_frame[y:y+h, x:x+w]
                    input_face = detector.preprocess_face(face_img)
                    pred = detector.model.predict(input_face, verbose=0)[0]
                    idx = np.argmax(pred)
                    label = detector.labels[idx]
                    confidence = pred[idx] * 100
                    status = "✓ SAFE" if label == "With Mask" else "⚠ NO MASK"
                    print(f"   • Face {i}: {label} ({confidence:.1f}%) → {status} at bbox ({x},{y},{w},{h})")
            else:
                print(f"❌ No faces detected in '{fname}'.")

            print("\n" + "="*60 + "\n")

        except Exception as e:
            print(f"❌ Unable to process '{fname}': {e}")
            continue

# ============================================
# 4) Usage instructions
# ============================================
print("\n🚀 Ready to use!")
print(" - To capture one webcam frame and detect masks:  detector.capture_and_detect()")
print(" - To upload local images and detect masks:   upload_images_and_detect()\n")


✅ Model loaded: /content/mask_model.h5, input size: 160x160

🚀 Ready to use!
 - To capture one webcam frame and detect masks:  detector.capture_and_detect()
 - To upload local images and detect masks:   upload_images_and_detect()



In [ ]:
# ============================================
# (New Cell) Initialize the mask detector
# ============================================

# If you used Cell 2’s files.upload(), then `model_filename` already holds the .h5 name.
# Otherwise, replace 'mask_detector_model.h5' with your actual filename.

detector = ColabMaskDetector("mask_model.h5")


✅ Model loaded: mask_model.h5, input size: 160x160


In [ ]:
# ============================================
# CELL X: Upload Images from Local System and Detect Masks
# ============================================

from google.colab import files
from PIL import Image

def upload_images_and_detect():
    """
    Upload one or more image files from your system, then run face+mask detection
    on each uploaded image and display results side by side (original vs annotated).
    """
    # Prompt user to upload
    print("📁 Please select one or more image files to upload (e.g. .jpg, .png):")
    uploaded_dict = files.upload()
    if not uploaded_dict:
        print("❌ No files uploaded.")
        return

    for fname, file_bytes in uploaded_dict.items():
        try:
            # Load the uploaded image into a PIL Image
            img = Image.open(io.BytesIO(file_bytes)).convert("RGB")
            img_array = np.array(img)

            # Run face+mask detection
            annotated_img, results = detector.detect_faces_and_masks(img_array)

            # Display original and annotated side by side
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
            ax1.imshow(img_array)
            ax1.set_title(f"Original: {fname}", fontsize=14)
            ax1.axis("off")

            ax2.imshow(annotated_img)
            ax2.set_title(f"Mask Detection: {fname}", fontsize=14)
            ax2.axis("off")

            plt.tight_layout()
            plt.show()

            # Print a simple summary
            if results:
                print(f"✅ {len(results)} face(s) detected in '{fname}':")
                for i, r in enumerate(results, start=1):
                    label, conf = r["label"], r["confidence"]
                    status = "✓ SAFE" if label == "With Mask" else "⚠ NO MASK"
                    bbox = r["bbox"]
                    print(f"   • Face {i}: {label} ({conf*100:.1f}%) → {status} at {bbox}")
            else:
                print(f"❌ No faces detected in '{fname}'.")

            print("\n" + "="*60 + "\n")

        except Exception as e:
            print(f"❌ Unable to process '{fname}': {e}")
            continue

# Usage: after adding this cell, run:
# upload_images_and_detect()


In [ ]:
detector.capture_and_detect()

📸 Waiting for webcam... Click 'Capture' when ready.


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [1]:
upload_images_and_detect()

NameError: name 'upload_images_and_detect' is not defined